# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

<b>Dependencies and Setup</b>

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

<b>Import API key</b>

In [2]:
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_file = "../WeatherPy/WeatherOutput/city_weather_main.csv"

In [4]:
city_weather_df = pd.read_csv(weather_file)
city_weather_df.head()

,City,Country,Latitude,Longitude,Cloudiness,Date,Wind,Humidity,Max Temp
0,Ushuaia,AR,-54.80,-68.30,75,1608321285,5.75,66,50.00
1,Qaanaaq,GL,77.48,-69.36,88,1608321358,2.51,87,0.88
2,San Vicente,AR,-26.62,-54.13,44,1608321733,2.13,40,89.02
3,Yellowknife,CA,62.46,-114.35,20,1608321330,3.36,64,-41.80
4,East London,ZA,-33.02,27.91,100,1608321491,12.66,90,70.07


### Humidity Heatmap

<b>Configure gmaps</b>

In [5]:
gmaps.configure(api_key=g_key)

<b>Use the Lat and Lng as locations and Humidity as the weight.</b>

In [6]:
locations = city_weather_df[['Latitude', 'Longitude']]

rating = city_weather_df['Humidity'].astype(float)

<b>Add Heatmap layer to map</b>

In [8]:
# Adding Heatmap coding

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

<b>Narrow down the cities to fit weather conditions.</b>

In [9]:
# I used the given conditions of between 70 & 80 degrees, less the 10 mph and 0& humidity.

perfect_weather_df = city_weather_df

perfect_weather_df = perfect_weather_df.loc[(perfect_weather_df["Max Temp"] < 80) &
                                           (perfect_weather_df['Max Temp'] > 70)]

perfect_weather_df = perfect_weather_df.loc[perfect_weather_df["Wind"] < 10]
                                            
perfect_weather_df = perfect_weather_df.loc[perfect_weather_df["Cloudiness"] == 0]
                                            
perfect_weather_df

,City,Country,Latitude,Longitude,Cloudiness,Date,Wind,Humidity,Max Temp
67,Saint-Philippe,RE,-21.36,55.77,0,1608321358,2.24,83,75.2
81,Pisco,PE,-13.70,-76.22,0,1608321267,9.17,69,73.4
188,Dakar,SN,14.69,-17.44,0,1608321824,9.17,88,71.6
274,Alibag,IN,18.64,72.88,0,1608321779,3.36,69,78.8
282,Oussouye,SN,12.48,-16.55,0,1608321872,4.70,57,78.8
317,Saint-Joseph,RE,-21.37,55.62,0,1608321878,2.24,83,75.2
495,Saint-Pierre,RE,-21.34,55.48,0,1608321973,2.24,83,75.2


<b>Drop any rows will null values.</b>

In [10]:
# Skipping as there are no null columns

### Hotel Map

<b>Store into variable named `hotel_df`.</b>

In [11]:
# DataFrame should need not change other than the name

hotel_df = perfect_weather_df
hotel_df

,City,Country,Latitude,Longitude,Cloudiness,Date,Wind,Humidity,Max Temp
67,Saint-Philippe,RE,-21.36,55.77,0,1608321358,2.24,83,75.2
81,Pisco,PE,-13.70,-76.22,0,1608321267,9.17,69,73.4
188,Dakar,SN,14.69,-17.44,0,1608321824,9.17,88,71.6
274,Alibag,IN,18.64,72.88,0,1608321779,3.36,69,78.8
282,Oussouye,SN,12.48,-16.55,0,1608321872,4.70,57,78.8
317,Saint-Joseph,RE,-21.37,55.62,0,1608321878,2.24,83,75.2
495,Saint-Pierre,RE,-21.34,55.48,0,1608321973,2.24,83,75.2


<b>Add a "Hotel Name" column to the DataFrame.</b>

In [12]:
# Created blank column for the hotel names

hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Latitude,Longitude,Cloudiness,Date,Wind,Humidity,Max Temp,Hotel Name
67,Saint-Philippe,RE,-21.36,55.77,0,1608321358,2.24,83,75.2,
81,Pisco,PE,-13.70,-76.22,0,1608321267,9.17,69,73.4,
188,Dakar,SN,14.69,-17.44,0,1608321824,9.17,88,71.6,
274,Alibag,IN,18.64,72.88,0,1608321779,3.36,69,78.8,
282,Oussouye,SN,12.48,-16.55,0,1608321872,4.70,57,78.8,
317,Saint-Joseph,RE,-21.37,55.62,0,1608321878,2.24,83,75.2,
495,Saint-Pierre,RE,-21.34,55.48,0,1608321973,2.24,83,75.2,


<b>Set parameters to search for hotels with 5000 meters.</b>

In [13]:
params = {'radius': 5000, 'types': 'lodging', 'key': g_key, 'keyword': 'hotel'}

<b>Hit the Google Places API for each city's coordinates.</b>

In [14]:
# Creating for loop to gather API informatation

for index, row in hotel_df.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    city = row['City']
    
    params['location'] = f'{latitude},{longitude}'
    
    google_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(google_url, params=params).json()
    
    hotel_data = response['results']
    hotel_name = hotel_data[0]['name']
    
    try:
        print(f'The closest hotel to {city} is {hotel_name}.')
        #print(f"The Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = hotel_data[0]['name']

    except:
        print("Missing data...Skipping.")
        
    print('------------------------------')
    
    #hotel_data
        

The closest hotel to Saint-Philippe is Les Embruns Du Baril.
------------------------------
The closest hotel to Pisco is HOSTAL MIRAMAR.
------------------------------
The closest hotel to Dakar is Pullman Dakar Teranga.
------------------------------
The closest hotel to Alibag is Radisson Blu Resort & Spa - Alibaug, India.
------------------------------
The closest hotel to Oussouye is Auberge du Routard.
------------------------------
The closest hotel to Saint-Joseph is Village Sauvage.
------------------------------
The closest hotel to Saint-Pierre is Le Battant Des Lames.
------------------------------


In [15]:
#Checking DataFrame to make sure hotels added

hotel_df

,City,Country,Latitude,Longitude,Cloudiness,Date,Wind,Humidity,Max Temp,Hotel Name
67,Saint-Philippe,RE,-21.36,55.77,0,1608321358,2.24,83,75.2,Les Embruns Du Baril
81,Pisco,PE,-13.70,-76.22,0,1608321267,9.17,69,73.4,HOSTAL MIRAMAR
188,Dakar,SN,14.69,-17.44,0,1608321824,9.17,88,71.6,Pullman Dakar Teranga
274,Alibag,IN,18.64,72.88,0,1608321779,3.36,69,78.8,"Radisson Blu Resort & Spa - Alibaug, India"
282,Oussouye,SN,12.48,-16.55,0,1608321872,4.70,57,78.8,Auberge du Routard
317,Saint-Joseph,RE,-21.37,55.62,0,1608321878,2.24,83,75.2,Village Sauvage
495,Saint-Pierre,RE,-21.34,55.48,0,1608321973,2.24,83,75.2,Le Battant Des Lames


<b>NOTE: Do not change any of the code in this cell<br>
Using the template add the hotel marks to the heatmap</b>

In [16]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

<b>Add marker layer on top of heat map</b>

In [19]:
fig = gmaps.figure(map_type="HYBRID")
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(markers)

<b>Display figure.</b>

In [20]:
fig

Figure(layout=FigureLayout(height='420px'))